# Maximum Likelihood Estimation

In [ ]:
import os
import argparse
import time
from datetime import datetime
import random
import sys
import atiSetup

from fit import runFits

In [ ]:
############## SET ENVIRONMENT VARIABLES ##############
REPO_HOME = os.environ['REPO_HOME']
USE_MPI, USE_GPU, RANK_MPI = atiSetup.setup(globals())

cfgfile = f'{REPO_HOME}/tests/samples/SIMPLE_EXAMPLE/fitInit.cfg'
randomSeed = int(time.time())
seedfile = f'seed_file'
numRnd = 0

In [ ]:
print("\n\n === COMMANDLINE ARGUMENTS === ")
print("Config file:", cfgfile)
print("Seed file:", seedfile)
print("Number of random fits:", numRnd)
print("Random seed:", randomSeed)
print(" ============================= \n\n")

We first load the configuration file by first passing it through a parser. We can display a detailed representation of its contents like so

In [ ]:
parser = ConfigFileParser(cfgfile)
cfgInfo: ConfigurationInfo = parser.getConfigurationInfo()
cfgInfo.display()

The confirguation file specifies a datareader called `ROOTDataReader` and the amplitude called `Zlm`. There are many datareaders (i.e. for `FSRoot` and another for `Bootstrapping`) and amplitudes (i.e. `BreitWigner` and `Piecewise`). Below, `DataReader` is an alias for `ROOTDataReader` or its `MPI` implementation. These aliases are created by `atiSetup`.

In [ ]:
AmpToolsInterface.registerAmplitude( Zlm() )
AmpToolsInterface.registerDataReader( DataReader() )
# AmpToolsInterface.registerAmplitude( BreitWigner() )
# AmpToolsInterface.registerAmplitude( Piecewise() )
# AmpToolsInterface.registerDataReader( DataReaderBootstrap() )

`AmpToolsInterface` is the primary way to interact with the `AmpTools` minimization library. From this `object` we can access things like a `ParameterManager` and a `MinimizationManager`. 

In [ ]:
ati = AmpToolsInterface( cfgInfo )
AmpToolsInterface.setRandomSeed(randomSeed)

A `fit` module is defined in PyAmpTools to run the standard MLE fit. We simply pass the recently created `ati` instance and the number of fits with random initializations to perform. Randomization is beneficial as the optimizer could get stuck in unwanted local minima.

A single fit is performed without randomization is performed if `numRnd=0`

See [fit](https://lan13005.github.io/PyAmpTools/api/fit.html) for API and source code, including `runFits` below

In [ ]:
nll = runFits(ati, N=numRnd, USE_MPI=USE_MPI)

After the fit is complete, a `.fit` file will be created with the name defined by the `fit` keyword in the cfg file. This file contains the results of the fit. A tutorial on how to plot the results of these `.fit` files will be shown in 

In [ ]:
!rm -f seed_0.txt
!rm -f normint
!rm -f result_0.fit